In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [2]:
import ee 
import geemap

ee.Initialize(project="thurgau-irrigation")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
from src.et_blue.exporting_utils import postprocess_et_blue_raw
from src.et_blue.compute_et_blue import compute_et_blue, compute_volumetric_et_blue, postprocess_et_blue

from src.et_green.compute_et_green import calculate_band_std_dev

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
)

from utils.date_utils import print_collection_dates

---

## Constants

In [4]:
YEAR = 2023

PATH_TO_AOI = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"
PATH_TO_ET_GREEN = f"projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_{YEAR}_dekadal_from_WaPOR_10m"
PATH_TO_ET_BLUE_RAW = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw/ET_blue_raw_{YEAR}_dekadal_from_WaPOR_10m"

ET_GREEN_BAND_NAME = "ET_green"
SCALING_FACTOR = 100  # scaling factor for the ET band to convert it back to float
NUMBER_OF_IMAGES = 21
TEMPORAL_RESOLUTION = "dekadal"


ET_BLUE_POSTPROCESSED_PATH = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed/ET_blue_postprocessed_{YEAR}_{TEMPORAL_RESOLUTION}_from_WaPOR_10m"
EXPORT_IMAGE_RESOLUTION = 10  # in meters

## 1. Load assets

In [5]:
aoi = ee.FeatureCollection(PATH_TO_AOI).geometry().simplify(500).buffer(100)

In [6]:
et_green_collection = (
    ee.ImageCollection(PATH_TO_ET_GREEN)
    .filterBounds(aoi)
    .filterDate(f"{YEAR}-04-01", f"{YEAR}-10-31")
    .map(lambda img: back_to_float(img, SCALING_FACTOR))
)

print_collection_dates(et_green_collection)


et_green_list = et_green_collection.toList(et_green_collection.size())

Dates of images in the collection:
2023-04-01
2023-04-11
2023-04-21
2023-05-01
2023-05-11
2023-05-21
2023-06-01
2023-06-11
2023-06-21
2023-07-01
2023-07-11
2023-07-21
2023-08-01
2023-08-11
2023-08-21
2023-09-01
2023-09-11
2023-09-21
2023-10-01
2023-10-11
2023-10-21


In [7]:
et_blue_raw_collection = (
    ee.ImageCollection(PATH_TO_ET_BLUE_RAW)
    .filterBounds(aoi)
    .filterDate(f"{YEAR}-04-01", f"{YEAR}-10-31")
    .map(lambda img: back_to_float(img, SCALING_FACTOR))
)

print_collection_dates(et_blue_raw_collection)


et_blue_raw_list = et_blue_raw_collection.toList(et_blue_raw_collection.size())

Dates of images in the collection:
2023-04-01
2023-04-11
2023-04-21
2023-05-01
2023-05-11
2023-05-21
2023-06-01
2023-06-11
2023-06-21
2023-07-01
2023-07-11
2023-07-21
2023-08-01
2023-08-11
2023-08-21
2023-09-01
2023-09-11
2023-09-21
2023-10-01
2023-10-11
2023-10-21


## 2. Computing and exporting ET blue postprocessed

In [8]:
postprocess_et_blue_raw(
    et_blue_raw_list=et_blue_raw_list,
    et_green_list=et_green_list,
    year=YEAR,
    aoi=aoi,
    asset_path=ET_BLUE_POSTPROCESSED_PATH,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=EXPORT_IMAGE_RESOLUTION,
    et_green_band_name=ET_GREEN_BAND_NAME,
)

Exporting ET_blue_postprocessed_dekadal_2023_04_D1 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed/ET_blue_postprocessed_2023_dekadal_from_WaPOR_10m/ET_blue_postprocessed_dekadal_2023_04_D1
Exporting ET_blue_postprocessed_dekadal_2023_04_D2 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed/ET_blue_postprocessed_2023_dekadal_from_WaPOR_10m/ET_blue_postprocessed_dekadal_2023_04_D2
Exporting ET_blue_postprocessed_dekadal_2023_04_D3 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed/ET_blue_postprocessed_2023_dekadal_from_WaPOR_10m/ET_blue_postprocessed_dekadal_2023_04_D3
Exporting ET_blue_postprocessed_dekadal_2023_05_D1 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed/ET_blue_postprocessed_2023_dekadal_from_WaPOR_10m/ET_blue_postprocessed_dekadal_2023_05_D1
Exporting ET_blue_postprocessed_dekadal_2023_05_D2 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_postproces

## 3. Sanity check

In [9]:
et_blue_postprocessed = []

et_blue_previous = None

for i in range(NUMBER_OF_IMAGES):
    # Get current images
    et_green = ee.Image(et_green_list.get(i))
    et_blue_present = ee.Image(et_blue_raw_list.get(i))

    # Initialize previous for first iteration
    if et_blue_previous is None:
        et_blue_previous = et_blue_present

    # Calculate threshold from ET green
    threshold = calculate_band_std_dev(et_green, ET_GREEN_BAND_NAME)

    # Post process using the previous processed image
    et_blue = postprocess_et_blue(et_blue_present, et_blue_previous, threshold)

    # Compute and add volumetric band
    et_blue_m3 = compute_volumetric_et_blue(et_blue)
    et_blue = et_blue.addBands(et_blue_m3)

    # Store current processed image for next iteration
    et_blue_previous = et_blue.select("ET_blue")

    et_blue_postprocessed.append(et_blue)

In [10]:
image = ee.Image(et_blue_postprocessed[5])

Map = geemap.Map()

vis_params = {
    "bands": ["ET_blue"],
    "min": 0,
    "max": 1,
    "palette": ["blue", "green", "yellow", "red"],
}

Map.addLayer(image, vis_params, "ET_blue")

# Map.addLayer(aoi, {}, "AOI")
Map.center_object(aoi, 12)

Map

Map(center=[47.56858787382066, 9.092720596553875], controls=(WidgetControl(options=['position', 'transparent_b…